# spaCy para análise de texto

O spaCy é Framework de Processamento de linguagem natural (NLP).

NLP = analisar linguagem humana através de um sistema de computador, também chamado de linguística computacional.

Com spaCy podemos analisar as estruturas textuais, classes gramaticais, reconhecer entidades, analisar as relações entre os termos, etc.


O que vamos aprender hoje:

* o que é o spaCy;
* instalar, carregar modelos
* Criar um `Doc Container`
* Trabalhar com sentenças
* Entender o que é um `token` e seus atributos
* Aplicar reconhecimento de entidades.

## Instalando o spaCy

In [ ]:
!pip install spacy

## Carregando modelos

In [ ]:
import spacy


In [ ]:
!python -m spacy download pt_core_news_sm

In [ ]:
# carregar o modelo que foi previamente instalado, nesse caso 
# o modelo 'pt_core_news_sm' = modelo de português reduzido
nlp = spacy.load('pt_core_news_sm')

### O que são esses modelos?

Ver a [documentação do spaCy](https://spacy.io/models/pt)

## Criar um `Doc Container`

In [ ]:
# ler o texto
text = open('wlamyra.txt', 'r').read()

In [ ]:
# processar o texto
doc = nlp(text)

## Trabalhar com sentenças

In [ ]:
# mostrar sentenças
doc.sents


In [ ]:
# ver as sentenças do texto
print([sent.text for sent in doc.sents])


In [ ]:
doc.sents[10].text

In [ ]:
# transformar o objeto doc.sents em uma lista de strings
sents = list(doc.sents)

In [ ]:
doc.sents[10].text

## Tokens

In [ ]:
print (len(doc))
print (len(text))

In [ ]:
# imprimir os 10 primeiros tokens
for token in doc[:10]:
    print(token)

In [ ]:
# e se fizermos isso com a variável text?
for i in text[:10]:
    print (i)

### Atributos de um token

* .text
* .head
* .left_edge
* .right_edge
* .ent_type_
* .iob_
* .lemma_
* .morph
* .pos_
* .dep_
* .lang_


In [ ]:
# vamos utilizar um token da terceira sentença 
sent5 = sents[4]
sent5

In [ ]:
len(sent5)

In [ ]:
token1 = sent5[30]
token1

In [ ]:
# head = "pai" sintático do token1
token1.head

In [ ]:
# entity type do token1
token1.ent_type

In [ ]:
token1.ent_type_

In [ ]:
# Lemma =  base lematizada do token1 (reduzir à base da palavra)
token2 = sent5[28]
print(token2.text, token2.lemma_)

In [ ]:
# Análise morfológica
print(token2.text, token2.morph)

In [ ]:
# Relação de dependência sintática
print(token1.text, token1.dep_)
print(token2.text, token2.dep_)

In [ ]:
# Classe gramatical = Part of Speech
print(token1.text, token1.pos_)
print(token2.text, token2.pos_)


In [ ]:
print(token1.text, token1.tag)

Para ver todas as características linguísticas possíveis dos spaCy, veja o [link](https://spacy.io/usage/linguistic-features).

### Visualizar as relações entre tokens

In [ ]:
from spacy import displacy

In [ ]:
displacy.render(sent5, style="dep", minify=True, options={'distance': 110, 'compact': True})

## Entidades (Named Entities Recognition)

In [ ]:
# imprimir as entidades do texto
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
displacy.render(doc, style="ent")

### Entidades Nomeadas do spaCy

Tipo de etiqueta|Descrição|
|:---:|:---:|
|PERSON|People, including fictional.|
|NORP|Nationalities or religious or political groups.|
|FAC|Buildings, airports, highways, bridges, etc.|
|ORG|Companies, agencies, institutions, etc.|
|GPE|Countries, cities, states.|
|LOC|Non-GPE locations, mountain ranges, bodies of water.|
|PRODUCT|Objects, vehicles, foods, etc. (Not services.)|
|EVENT|Named hurricanes, battles, wars, sports events, etc.|
|WORK_OF_ART|Titles of books, songs, etc.|
|LAW|Named documents made into laws.|
|LANGUAGE|Any named language.|
|DATE|Absolute or relative dates or periods.|
|TIME|Times smaller than a day.|
|PERCENT|Percentage, including ”%“.|
|MONEY|Monetary values, including unit.|
|QUANTITY|Measurements, as of weight or distance.|
|ORDINAL|“first”, “second”, etc.|
|CARDINAL|Numerals that do not fall under another type.|

In [ ]:
# imprimiar apenas as entidades do tipo PERSON
for named_entity in doc.ents:
    if named_entity.label_ == "PER":
        print(named_entity)

In [ ]:
# imprimir apenas as entidades do tipo VERB e seu lemma
for token in doc:
    if token.pos_ == 'VERB':
        print(token, token.lemma_)

## Vamos ver como funciona em inglês?

In [ ]:
corpus = open('pos_1900.txt', 'r').read()

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
nlp_en = spacy.load('en_core_web_sm')

In [ ]:
doc_en = nlp_en(corpus)

In [ ]:
displacy.render(doc_en, style="ent")

### Encontrar as pessoas e contar

<small>Inspirado no exemplo de Melanie Walsh no curso [Introduction to cultural analytics & Python](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/Multilingual/Portuguese/02-Named-Entity-Recognition-Portuguese.html#get-people)</small>

In [ ]:
import pandas as pd
from collections import Counter # para contar o número de ocorrências de cada palavra


In [ ]:
# criar conjunto de textos divididos por quebra de linha
chunked_text = corpus.split('\n')
# passa a lista de textos para o pipeline de análise do Spacy
chunked_documents = list(nlp_en.pipe(chunked_text))

In [ ]:
len(chunked_documents)

In [ ]:
# selecionar todas os tokens com a classe POS 'PERSON' e armazenar em uma lista
people = []
for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == "PERSON":
            people.append(named_entity.text)

# contar o número de ocorrências de cada palavra
people_count = Counter(people)

# criar um dataframe com as palavras e seus respectivos números de ocorrências
df_people = pd.DataFrame(people_count.most_common(), columns=['character', 'count'])
df_people

In [ ]:
# selecionar todas os tokens com a classe POS 'GPE' ou 'LOC' e armazenar em uma lista
places = []
for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == 'GPE' or  named_entity.label_ == 'LOC':
            places.append(named_entity.text)

places_count = Counter(places)

df_places = pd.DataFrame(places_count.most_common(), columns=['place', 'count'])
df_places